In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin
import time

driver = webdriver.Chrome()
driver.get("https://gravitas.acr.org/acportal")
WebDriverWait(driver, 10).until(lambda d: d.execute_script("return document.readyState") == "complete")

# Step 1: Get all <a> inside .spanResolutionDocTitle
link_elements = driver.find_elements(By.CSS_SELECTOR, '.spanResolutionDocTitle a')

# Step 2: Get full URLs from hrefs
base_url = "https://gravitas.acr.org/acportal"  # Replace with your real base domain
links = [urljoin(base_url, a.get_attribute("href")) for a in link_elements]

In [5]:
len(links)

3890

In [ ]:
import requests, pandas as pd
from bs4 import BeautifulSoup

#TEST SET BEFORE FULL SCRAPE
links_trunc = links[0:5]

/Users/satrio/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [8]:
links_trunc

['https://gravitas.acr.org/ACPortal/GetDataForOneScenario?senarioId=3209',
 'https://gravitas.acr.org/ACPortal/GetDataForOneScenario?senarioId=5930',
 'https://gravitas.acr.org/ACPortal/GetDataForOneScenario?senarioId=5929',
 'https://gravitas.acr.org/ACPortal/GetDataForOneScenario?senarioId=5940',
 'https://gravitas.acr.org/ACPortal/GetDataForOneScenario?senarioId=5939']

In [ ]:
def scrape_scenario(scenario_url: str) -> list[dict]:
    """Return a list of dictionaries—one per procedure row."""
    html = requests.get(scenario_url, timeout=10).text
    soup = BeautifulSoup(html, "lxml")

    variant_li = soup.select_one('a[href^="#variant"]')  
    variant = variant_li.get_text(strip=True) if variant_li else ""

    table = soup.select_one("table.tblResDocs")
    rows = []
    if not table:
        return rows                        

    curr_scenario = curr_id = None

    for tr in table.select("tbody > tr"):
        tds = tr.find_all("td", recursive=False)

        if len(tds[0].find_all("span")) >= 1 and tds[0]["rowspan"]:
            curr_scenario = tds[0].get_text(strip=True)
            curr_id       = tds[1].get_text(strip=True)

            proc_td, adult_td, peds_td, cat_td = tds[2:6]
        else:
            proc_td, adult_td, peds_td, cat_td = tds[0:4]

        rows.append({
            "variant"                : variant,
            "scenario"               : curr_scenario,
            "scenario_id"            : curr_id,
            "procedure"              : proc_td.get_text(" ", strip=True),
            "adult_rrl"              : adult_td.get_text(" ", strip=True),
            "peds_rrl"               : peds_td.get_text(" ", strip=True),
            "appropriateness_category": cat_td.get_text(" ", strip=True)
        })

    return rows

In [ ]:
BASE = "https://gravitas.acr.org"

url = urljoin(BASE, "/ACPortal/GetDataForOneScenario?senarioId=7149")
data = scrape_scenario(url)



df = pd.DataFrame(data)
print(df.head())

                                             variant  \
0  Suspected vascular steal syndrome (upper or lo...   
1  Suspected vascular steal syndrome (upper or lo...   
2  Suspected vascular steal syndrome (upper or lo...   
3  Suspected vascular steal syndrome (upper or lo...   
4  Suspected vascular steal syndrome (upper or lo...   

                                            scenario scenario_id  \
0  Vascular steal syndrome suspected, upper extre...     3196520   
1  Vascular steal syndrome suspected, upper extre...     3196520   
2  Vascular steal syndrome suspected, upper extre...     3196520   
3  Vascular steal syndrome suspected, upper extre...     3196520   
4  Vascular steal syndrome suspected, upper extre...     3196520   

                                           procedure       adult_rrl  \
0  Fluoroscopy fistulography hemodialysis access ...    1-10 mSv ☢☢☢   
1  US duplex Doppler hemodialysis access upper ex...         0 mSv O   
2               CTA upper extremity wi

In [23]:
df

,variant,scenario,scenario_id,procedure,adult_rrl,peds_rrl,appropriateness_category
0,Suspected vascular steal syndrome (upper or lo...,"Vascular steal syndrome suspected, upper extre...",3196520,Fluoroscopy fistulography hemodialysis access ...,1-10 mSv ☢☢☢,,Usually appropriate
1,Suspected vascular steal syndrome (upper or lo...,"Vascular steal syndrome suspected, upper extre...",3196520,US duplex Doppler hemodialysis access upper ex...,0 mSv O,0 mSv [ped] O,Usually appropriate
2,Suspected vascular steal syndrome (upper or lo...,"Vascular steal syndrome suspected, upper extre...",3196520,CTA upper extremity with IV contrast,1-10 mSv ☢☢☢,,May be appropriate
3,Suspected vascular steal syndrome (upper or lo...,"Vascular steal syndrome suspected, upper extre...",3196520,CTV upper extremity with IV contrast,10-30 mSv ☢☢☢☢,,Usually not appropriate
4,Suspected vascular steal syndrome (upper or lo...,"Vascular steal syndrome suspected, upper extre...",3196520,MRA upper extremity without and with IV contrast,0 mSv O,0 mSv [ped] O,Usually not appropriate
5,Suspected vascular steal syndrome (upper or lo...,"Vascular steal syndrome suspected, upper extre...",3196520,MRA upper extremity without IV contrast,0 mSv O,0 mSv [ped] O,Usually not appropriate
6,Suspected vascular steal syndrome (upper or lo...,"Vascular steal syndrome suspected, upper extre...",3196520,MRV upper extremity without and with IV contrast,0 mSv O,0 mSv [ped] O,Usually not appropriate
7,Suspected vascular steal syndrome (upper or lo...,"Vascular steal syndrome suspected, upper extre...",3196520,MRV upper extremity without IV contrast,0 mSv O,0 mSv [ped] O,Usually not appropriate


In [28]:
df = pd.DataFrame(list)

for link in links_trunc:
    data = scrape_scenario(link)
    df_temp = pd.DataFrame(data)
    df = pd.concat([df, df_temp])

In [29]:
df

,variant,scenario,scenario_id,procedure,adult_rrl,peds_rrl,appropriateness_category
0,Male of any age with physical examination susp...,"Axillary adenopathy, breast cancer suspected",3110141,US breast,0 mSv O,0 mSv [ped] O,Usually appropriate
1,Male of any age with physical examination susp...,"Axillary adenopathy, breast cancer suspected",3110141,Digital breast tomosynthesis diagnostic,0.1-1mSv ☢☢,,Usually appropriate
2,Male of any age with physical examination susp...,"Axillary adenopathy, breast cancer suspected",3110141,Mammography diagnostic,0.1-1mSv ☢☢,,Usually appropriate
3,Male of any age with physical examination susp...,"Axillary adenopathy, breast cancer suspected",3110141,MRI breast without and with IV contrast,0 mSv O,0 mSv [ped] O,Usually not appropriate
4,Male of any age with physical examination susp...,"Axillary adenopathy, breast cancer suspected",3110141,MRI breast without IV contrast,0 mSv O,0 mSv [ped] O,Usually not appropriate
0,"Female. New palpable, bilateral, axillary lump...","Axillary lump, palpable, new, bilateral, initi...",3194817,US axilla,0 mSv O,0 mSv [ped] O,Usually appropriate
1,"Female. New palpable, bilateral, axillary lump...","Axillary lump, palpable, new, bilateral, initi...",3194817,Digital breast tomosynthesis diagnostic,0.1-1mSv ☢☢,,Usually not appropriate
2,"Female. New palpable, bilateral, axillary lump...","Axillary lump, palpable, new, bilateral, initi...",3194817,Mammography diagnostic,0.1-1mSv ☢☢,,Usually not appropriate
3,"Female. New palpable, bilateral, axillary lump...","Axillary lump, palpable, new, bilateral, initi...",3194817,MRI breast without and with IV contrast,0 mSv O,0 mSv [ped] O,Usually not appropriate
4,"Female. New palpable, bilateral, axillary lump...","Axillary lump, palpable, new, bilateral, initi...",3194817,MRI breast without IV contrast,0 mSv O,0 mSv [ped] O,Usually not appropriate


In [ ]:
import os
import pandas as pd
from tqdm import tqdm

csv_path = "acr_scenarios.csv"

first_time = not os.path.isfile(csv_path)

with open(csv_path, mode="a", newline="", encoding="utf-8") as f:
    for link in tqdm(links, desc="Scraping"):
        print(f"Processing link number ")
        try:
            rows = scrape_scenario(link)          
            if not rows:                         
                continue

            df_temp = pd.DataFrame(rows)
            df_temp.to_csv(
                f,                                
                index=False,
                header=first_time,               
                mode="a"
            )
            first_time = False                    

        except Exception as e:
            print(f"error on {link}: {e}")

Scraping:   0%|          | 0/3890 [00:00<?, ?it/s]

Processing link number 


Scraping:   0%|          | 1/3890 [00:00<40:51,  1.59it/s]

Processing link number 


Scraping:   0%|          | 2/3890 [00:01<45:07,  1.44it/s]

...

Scraping: 100%|█████████▉| 3888/3890 [46:47<00:01,  1.61it/s]

Processing link number 


Scraping: 100%|█████████▉| 3889/3890 [46:48<00:00,  1.56it/s]

Processing link number 


Scraping: 100%|██████████| 3890/3890 [46:48<00:00,  1.38it/s]
